<div style="text-align: center;"><br>
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Circuit Stitching

**Contents:**
* [GHZ State Preparation](#GHZ-State-Preparation)
* [Parallelization Across Qubit Register](#Parallelisation-Across-Qubit-Register)
* [Circuit Stitching](#Circuit-Stitching)
* [Conclusion](#Conclusion)

## GHZ State Preparation

In [11]:
from pytket.circuit import Circuit

n_qubits = 10
circ = Circuit(n_qubits)
circ.H(0)
for i in range(n_qubits-1):
    circ.CX(i,i+1);

In [12]:
from pytket.circuit import CircBox

ghz_subcircuits = [CircBox(circ.copy()) for _ in range(10)]

## Parallelization Across Qubit Register

In [15]:
from typing import List
import numpy as np
from pytket.circuit import (
    Circuit, 
    CircBox,
    Qubit
)

def parallelize_subcircuits(
    circbox_list: List[CircBox],
    device_qubits: int,
    n: int
) -> Circuit:
    r"""Parallelize subcircuit across device register.
    :param circbox_list: 
    :param_type: List[CircBox]
    :param device_qubits: 
    :param_type: int
    :param n: size of subcircuit register
    :param_type: int
    """
    p = np.floor_divide(device_qubits, n)
    qubits = [[Qubit(i) for i in range(q, q+n)] for q in range(0, device_qubits, n)]
    circuit = Circuit(device_qubits)
    for i in range(0, device_qubits+p, p):
        for j, (c, q_list) in enumerate(zip(circbox_list[i:i+p], qubits)):
            circuit.add_circbox(c, q_list)
            creg = circuit.add_c_register(f"creg_{i}_{j}", len(q_list))
            for q, cb in zip(q_list, creg):
                circuit.Measure(q, cb)
                circuit.Reset(q)
    return circuit

In [18]:
device_nqubits = 20
circuit1 = parallelize_subcircuits(ghz_subcircuits, device_nqubits, n_qubits)

In [19]:
from pytket.circuit.display import render_circuit_jupyter

render_circuit_jupyter(circuit1)

## Circuit Stitching

In [21]:
from pytket.circuit import Circuit

def stitch_circuit(
    circuit: Circuit, 
    s: int
) -> Circuit:
    stitched_circuit = Circuit(circuit.n_qubits)
    box = CircBox(circuit)
    for i in range(s):
        creg = stitched_circuit.add_c_register(f"creg_{i}", circuit.n_bits)
        stitched_circuit.add_circbox(box, stitched_circuit.qubits + list(creg))
    return stitched_circuit

In [22]:
circuit2 = stitch_circuit(circuit1, 40)

## Job Submission

In [24]:
from pytket.extensions.quantinuum import QuantinuumBackend

backend = QuantinuumBackend(device_name="H1-1E")
backend.login()

In [26]:
compiled_circuit = backend.get_compiled_circuit(circuit2, optimisation_level=2)

## Conclusion

<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>